# Betamaps GLM

Here, beta maps for individual trials are calculated for all subjects and both task conditions. For each trial separate GLM model is created with regressors:  
- `out_individual`: modelling sigle trial
- `out_perr_pos`: modelling remaining trials with positive PE
- `out_perr_neg`: modelling remaining trials with negative PE
- `dec_ons`, `dec_miss`, `dec_wcor`, `res_lbp`, `res_rbp`, `res_miss`: remaining task regressors modelling events of no interest 

Script implements single-trial-versus-other-trials method ([Mumford et al., 2012](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3251697/)). All regressors assume events having duration of 0 seconds. EPI images are masked before fitting GLM. GLM uses SPM hemodynamic response function, high pass filtering of 1/128Hz, standarization and 6mm smoothing kernel. Output beta-maps are raw beta values (voxelwise effect-size) for `out_individual` regressor. Beta maps for all trials are concatenated and stored as `nii.gz` file named according to BIDS-like convention:

> `betamaps/sub-<subject_label>_task-<taks_label>_betamaps.nii.gz`

Each `.nii.gz` file has shape $n_{epiX}\times n_{epiY}\times n_{epiZ}\times n_{trials}$

> Note that if you pass `design_matrices` argument to the `fit()` method of `FirstLevelModel` instance confounds **will no longer be used**! This lead to a bug, where images were not denoised even if `confounds` wass explicitely specified. No warning is raised from model instance. In this case you have to manually add confounds to your design matrix by concatenating DataFrames `dm` with `confounds`.

In [ ]:
import json
import os
from os.path import join
from pathlib import Path

import nibabel as nib
import numpy as np
import pandas as pd
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.glm_utils import Regressor, my_make_first_level_design_matrix
from dn_utils.path import path
from nistats.first_level_model import FirstLevelModel
from tqdm.notebook import tqdm

In [ ]:
# Create additional paths
path_modulations = join(path["nistats"], "modulations")
path_betamaps = join(path["bsc"], "betamaps")

Path(path_betamaps).mkdir(exist_ok=True, parents=True)

# Load behavioral data
beh, meta = load_behavioral_data(path=path["behavioral"], verbose=False)

# Load path to data files
with open(join(path["data_paths"], "fmri_filenames.json"), "r") as f:
    fmri_files = json.loads(f.read())
with open(join(path["data_paths"], "conf_filenames.json"), "r") as f:
    conf_files = json.loads(f.read())
with open(join(path["data_paths"], "mask_filenames.json"), "r") as f:
    mask_files = json.loads(f.read())    
    
# Load parametric modulations
modulations_wcor = np.load(join(path_modulations, "modulations_wcor.npy"))
modulations_exvl = np.load(join(path_modulations, "modulations_exvl.npy"))
modulations_perr = np.load(join(path_modulations, "modulations_perr.npy"))

### Setup GLM parameters

In [ ]:
# Times of image acquisition in seconds
n_scans = 730
n_trials = 110
t_r = 2
t_dec = 1.5
t_out = 1.5
frame_times = np.arange(n_scans) * t_r

# GLM parameters
glm_kwargs = {
    "t_r": 2,
    "hrf_model": "spm",
    "drift_model": "cosine",
    "noise_model": "ar1",
    "high_pass": 0.0078125, 
    "standardize": True,
    "smoothing_fwhm": 6,
    "n_jobs": -1,
}

# Only used to save as options
regressors_model = [
    "reg_dec_ons", "reg_dec_miss", "reg_dec_wcor", 
    "reg_res_lbp", "reg_res_rbp", "reg_res_miss", 
    "reg_out_individual", "reg_out_perr_pos", "reg_out_perr_neg"
]
regressors_save = ["reg_out_individual"]
confounds_regex = "trans|rot|csf|white_matter"

# Store settings in JSON file
with open(join(path_betamaps, "options_betamaps.json"), "w") as json_file:    
    json.dump(dict(
        regressors_model=regressors_model,
        regressors_save=regressors_save,
        confounds_regex=confounds_regex,
        **glm_kwargs
    ), json_file, sort_keys=False, indent=4)  

In [ ]:
for sub_idx, sub in enumerate(meta["dim1"]):
    for con_idx, con in enumerate(meta["dim2"]):
        
        print(f"Subject: {sub} | Condition: {con}")
        con_name = "prl" + con

        # Load subject data
        fmri_img = nib.load(fmri_files[con_name][sub_idx])
        confounds = pd.read_csv(conf_files[con_name][sub_idx], sep="\t")
        confounds = confounds.filter(regex=confounds_regex)
        confounds.index = frame_times
        
        # Setup events
        resp_type = beh[sub_idx, con_idx, :, meta["dim4"].index("response")]
        onset_dec = beh[sub_idx, con_idx, :, meta["dim4"].index("onset_dec")] 
        onset_res = beh[sub_idx, con_idx, :, meta["dim4"].index("onset_dec")] + \
                    beh[sub_idx, con_idx, :, meta["dim4"].index("rt")]
        onset_out = beh[sub_idx, con_idx, :, meta["dim4"].index("onset_out")]
        offset_dec = onset_dec + t_dec
        offset_out = onset_out + t_out

        # Calculate modulations
        modulation_wcor = modulations_wcor[sub_idx, con_idx, resp_type != 0]
        modulation_exvl = modulations_exvl[sub_idx, con_idx, resp_type != 0]
        modulation_perr = modulations_perr[sub_idx, con_idx]
        modulation_wcor_demeaned = modulation_wcor - np.mean(modulation_wcor)
        modulation_exvl_demeaned = modulation_exvl - np.mean(modulation_exvl)
        modulation_perr_demeaned = modulation_perr - np.mean(modulation_perr)

        # Decision phase regressors
        reg_dec_ons = Regressor("dec_ons", frame_times, 
                                onset_dec[resp_type != 0])
        reg_dec_miss = Regressor("dec_miss", frame_times, 
                                 onset_dec[resp_type == 0])
        reg_dec_wcor = Regressor("dec_wcor", frame_times, 
                                 onset_dec[resp_type != 0],
                                 modulation=modulation_wcor_demeaned)

        # Response phase regressors
        reg_res_lbp = Regressor("res_lbp", frame_times, 
                                onset_res[resp_type == -1])
        reg_res_rbp = Regressor("res_rbp", frame_times, 
                                onset_res[resp_type == 1])
        reg_res_miss = Regressor("res_miss", frame_times, 
                                 offset_dec[resp_type == 0])

        stat_maps = []

        for trial in tqdm(range(n_trials)):

            # Specify GLM
            fmri_glm = FirstLevelModel(
                **glm_kwargs,
                mask_img=nib.load(mask_files[con_name][sub_idx]),
                subject_label=sub,
            )
            
            # Split outcome events into three categories:
            #    trials_individual: 
            #        single separated trial
            #    trials_perr_pos: 
            #        remaining trials with positive PE
            #    trials_perr_neg: 
            #        remaining trials with negative PE
            won_bool = beh[sub_idx, con_idx, :, meta["dim4"].index("won_bool")]
            trials_individual = np.zeros((n_trials), dtype=bool)
            trials_perr_pos = won_bool.astype(bool)
            trials_perr_neg = ~trials_perr_pos
            trials_individual[trial] = True
            trials_perr_pos[trial] = False
            trials_perr_neg[trial] = False

            # Outcome phase regressors
            reg_out_individual = Regressor(
                "out_individual", frame_times, onset_dec[trials_individual])
            reg_out_perr_pos = Regressor(
                "out_perr_pos", frame_times, onset_out[trials_perr_pos])
            reg_out_perr_neg = Regressor(
                "out_perr_neg", frame_times, onset_out[trials_perr_neg])

            # Create design matrix
            regressors = [
                reg_dec_ons, reg_dec_miss, reg_dec_wcor, 
                reg_res_lbp, reg_res_rbp, reg_res_miss, 
                reg_out_individual, reg_out_perr_pos, reg_out_perr_neg
            ]
            dm, conditions = my_make_first_level_design_matrix(regressors)

            # !!! Avoid bug !!!
            # Add confounds to design matrix
            dm_full = design_matrices=pd.concat((dm, confounds), axis=1)
            dm_full = dm_full.fillna(0)

            # Fit GLM
            fmri_glm = fmri_glm.fit(fmri_img, design_matrices=dm_full)

            # Define contrast & compute statistical map
            stat_map = fmri_glm.compute_contrast(
                    np.hstack((conditions["out_individual"], 
                               np.zeros(confounds.shape[1]))),
                    stat_type=None,
                    output_type="effect_size"
            )
            stat_maps.append(stat_map)

        # Save concatenated betamaps
        betamaps_img = nib.concat_images(stat_maps)
        fname = f"sub-{sub}_task-prl{con}_betamaps.nii.gz"
        nib.save(betamaps_img, join(path_betamaps, fname))  